<h2>Generate dataset</h2>

In [1]:
import pandas as pd
import numpy as np
import time
import re
import pickle

from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm.notebook import tqdm

from utils import get_parallel_edges, get_weighted_edges

<h3>Read the raw data WITHOUT CLEANING</h3>

In [ ]:
fp = '/Users/victor/Desktop/CS/UST_summer/all_posts_clean_1.csv' # the directory of the raw data file, pipeliend from reddits

s_columns = ['id', 'author', 'created_utc', 'num_comments', 'clean_title', 'clean_selftext', 'num_title_emojis', 'num_text_emojis', 'title_length', 'text_length', 'score', 'num_replied_comments', 'num_replies_by_new', 'list_new_repliers', 'num_replies_by_old', 'list_old_repliers']

df = pd.read_csv(fp, low_memory=False, encoding='utf-8')[s_columns]

df.head() # test if it is read correctly

In [ ]:
# df = df[df.created_utc < 1590000000.0]

In [ ]:
start_time = time.time()
df.groupby('author').cumcount()
print("--- %s seconds ---" % (time.time() - start_time))

df["num_post"] = df.groupby('author').cumcount().to_numpy()
codes, uniques = pd.factorize(df["author"])
df["author_codes"] = codes

# df.to_csv('wsb_with_authorcode_num_post.csv')r

In [ ]:
def deleted_comments(left_bound, right_bound, dataframe):
    """_summary_

    Args:
        left_bound (_type_): _description_
        right_bound (_type_): _description_
        dataframe (_type_): _description_
    """
    indices = dataframe.index[(dataframe.created_utc >= left_bound) & (dataframe.created_utc <= right_bound)].tolist() # workout the total number of comments
    
    # print(indices)
    if len(indices) == 0:
        return 0
    
    unwanted_comments = 0
    for i in indices:
        if (dataframe['clean_selftext'].iloc[i] == 'deleted') | (dataframe['clean_selftext'].iloc[i] == 'removed'):
            unwanted_comments += 1
    
    return (unwanted_comments/len(indices))*100

In [ ]:
def replied_percent(left_bound, right_bound, dataframe):
    """_summary_

    Args:
        left_bound (_type_): _description_
        right_bound (_type_): _description_
        dataframe (_type_): _description_
    """
    indices = dataframe.index[(dataframe.created_utc >= left_bound) & (dataframe.created_utc <= right_bound)].tolist()
    
    negative_index = []
    negative_positive = 0
    
    for i in indices:
        if (dataframe['clean_selftext'].iloc[i] == 'deleted') | (dataframe['clean_selftext'].iloc[i] == 'removed'):
            negative_index.append(i)
    
    for i in negative_index:
        if dataframe.num_comments.iloc[i] > 0:
            negative_positive += 1
    
    negative_positive_rate = negative_positive / len(negative_index)
    
    positive_index = list(set(indices) - set(negative_index))
    positive_positive = 0
    
    for i in positive_index:
        if dataframe.num_comments.iloc[i] > 0:
            positive_positive += 1
    
    positive_positive_rate = positive_positive / len(positive_index)
    
    return (positive_positive_rate, negative_positive_rate)
        

---------------
<h3>Create an edge list</h3>

In [2]:
# get the edge list from an existing repo
fp_comment = '/Users/victor/Desktop/CS/UST_summer/complete_df_final.csv'
comment = pd.read_csv(fp_comment, low_memory = False)
fp_post = '/Users/victor/Desktop/CS/UST_summer/all_posts_clean_1.csv'
post = pd.read_csv(fp_post, low_memory = False)[['author', 'id', 'created_utc']]

In [5]:
parallel_edges = get_parallel_edges(comment, post)
weighted_edges = get_weighted_edges(parallel_edges)

In [ ]:
# save the edges lists
parallel_edges.to_csv('parallel_edges.csv')
weighted_edges.to_csv('weighted_edges.csv')